## Deliverable E, Part A: Preprocessing (Group 20)

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
import pandas.io.sql as sqlio
import psycopg2
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

pd.set_option('display.max_columns', None)

In [29]:
f = open(r"C:\Users\16136\Documents\passwordpy.txt", "r")
password = f.readline()
conn = psycopg2.connect(
    host = "www.eecs.uottawa.ca",
    port = 15432,
    user = "cbrad077",
    database = "group_20",
    password = password
)

In [30]:
features_query  = '''SELECT t1.*, 1 as SUM
FROM (SELECT r.holiday, r.weekend, p.age, p.gender, p.acquisition_group, p.outbreak_related, s.begin_date, s.end_date, s.city, s.zone_measures,  s.lockdown, s.stay_at_home_order, s.max_indoor_capacity, s.max_outdoor_capacity, d.*, w.*, SUM(f.fatal + f.resolved + f.unresolved)
FROM fact_table f, d_patient p, d_special_measures s, d_mobility d, d_weather w, d_reported_date r
WHERE f.patient_key = p.patient_surrogate_key AND f.special_measures_key = s.surrogate_key AND f.mobility_key = d.mobility_key AND f.weather_key = w.weather_key AND f.reported_date_key = r.reported_date_surrogate_key
GROUP BY r.holiday, r.weekend, p.age, p.gender, p.acquisition_group, p.outbreak_related, s.begin_date, s.end_date, s.city, s.zone_measures,  s.lockdown, s.stay_at_home_order, s.max_indoor_capacity, s.max_outdoor_capacity, d.mobility_key, w.weather_key) as t1, generate_series(1, t1.sum)'''

features_data = sqlio.read_sql_query(features_query, conn)
features_data = features_data.loc[:,~features_data.columns.duplicated()]
conn = None

In [31]:
features_data.head()

,holiday,weekend,age,gender,acquisition_group,outbreak_related,begin_date,end_date,city,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,mobility_key,subregion,province,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,weather_key,daily_high_temp,daily_low_temp,total_precipitation,station_name,daily_mean,weather_date,sum
0,False,False,20s,FEMALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,124,Toronto Division,Ontario,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,123,7.3,1.5,5.2,TORONTO INTL A,4.4,2020-10-27,11
1,False,False,20s,FEMALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,124,Toronto Division,Ontario,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,123,7.3,1.5,5.2,TORONTO INTL A,4.4,2020-10-27,11
2,False,False,20s,FEMALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,124,Toronto Division,Ontario,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,123,7.3,1.5,5.2,TORONTO INTL A,4.4,2020-10-27,11
3,False,False,20s,FEMALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,124,Toronto Division,Ontario,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,123,7.3,1.5,5.2,TORONTO INTL A,4.4,2020-10-27,11
4,False,False,20s,FEMALE,CC,False,2020-10-27,2020-11-14,Toronto,Orange,0,0,50,100,124,Toronto Division,Ontario,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,123,7.3,1.5,5.2,TORONTO INTL A,4.4,2020-10-27,11


In [32]:
# create pandas dataframe

features = pd.DataFrame(features_data, columns=["holiday", "weekend", "city", "daily_mean", 
                                                "total_precipitation", "zone_measures", "lockdown", "stay_at_home_order", 
                                                "max_indoor_capacity", "max_outdoor_capacity", "retail_and_recreation",
                                                "grocery_and_pharmacy", "parks", "transit_stations", "workplaces", 
                                                "residential", "age", "gender", "acquisition_group", "outbreak_related"])

features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
1,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
2,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
3,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
4,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,True,True,Toronto,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,MALE,OB,True
76452,True,True,Toronto,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,MALE,OB,True
76453,True,True,Ottawa,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,MALE,OB,True
76454,True,True,Ottawa,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,MALE,OB,True


In [33]:
# check for missing values 

features.isna().sum()

holiday                  0
weekend                  0
city                     0
daily_mean               0
total_precipitation      0
zone_measures            0
lockdown                 0
stay_at_home_order       0
max_indoor_capacity      0
max_outdoor_capacity     0
retail_and_recreation    0
grocery_and_pharmacy     0
parks                    0
transit_stations         0
workplaces               0
residential              0
age                      0
gender                   0
acquisition_group        0
outbreak_related         0
dtype: int64

In [34]:
# view value distribution

Counter(features["outbreak_related"])

Counter({False: 63503, True: 12953})

In [35]:
Counter(features["gender"])

Counter({'FEMALE': 38169,
         'GENDER DIVERSE': 15,
         'MALE': 37904,
         'UNSPECIFIED': 368})

In [36]:
Counter(features["city"])

Counter({'Toronto': 68532, 'Ottawa': 7924})

In [37]:
drop_index = features[features["gender"] == "UNSPECIFIED"].index
features.drop(drop_index, inplace = True)
drop_index = features[features["gender"] == "GENDER DIVERSE"].index
features.drop(drop_index, inplace = True)
drop_index = features[features["age"] == "UNKNOWN"].index
features.drop(drop_index, inplace = True)
features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
1,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
2,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
3,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
4,False,False,Toronto,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,FEMALE,CC,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,True,True,Toronto,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,MALE,OB,True
76452,True,True,Toronto,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,MALE,OB,True
76453,True,True,Ottawa,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,MALE,OB,True
76454,True,True,Ottawa,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,MALE,OB,True


In [38]:
# convert binary columns to 1/0 

features["outbreak_related"].replace({True: 1, False: 0}, inplace=True)
features["gender"].replace({"FEMALE": 1,"MALE": 0}, inplace=True)
features["city"].replace({"Ottawa": 1, "Toronto": 0}, inplace=True)
features["holiday"].replace({True: 1, False: 0}, inplace=True)
features["weekend"].replace({True: 1, False: 0}, inplace=True)

features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,1,CC,0
1,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,1,CC,0
2,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,1,CC,0
3,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,1,CC,0
4,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,20s,1,CC,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,1,1,0,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,0,OB,1
76452,1,1,0,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,<20,0,OB,1
76453,1,1,1,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,0,OB,1
76454,1,1,1,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,<20,0,OB,1


In [39]:
# fix age 

Counter(features["age"])

Counter({'20s': 15641,
         '30s': 12531,
         '40s': 10835,
         '50s': 11080,
         '60s': 7116,
         '70s': 3600,
         '80s': 2819,
         '90+': 1653,
         '<20': 10787})

In [40]:
# convert ages to normalized ordinals

features["age"].replace({'<20': 0.1, '20s': 0.2, '30s': 0.3, '40s': 0.4, 
                              '50s': 0.5, '60s': 0.6, '70s': 0.7, '80s': 0.8, '90+': 0.9}, inplace=True)
features

,holiday,weekend,city,daily_mean,total_precipitation,zone_measures,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,acquisition_group,outbreak_related
0,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,CC,0
1,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,CC,0
2,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,CC,0
3,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,CC,0
4,0,0,0,4.4,5.2,Orange,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,CC,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,1,1,0,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,0.1,0,OB,1
76452,1,1,0,0.7,0.0,Orange,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,0.1,0,OB,1
76453,1,1,1,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,0.1,0,OB,1
76454,1,1,1,-1.0,0.0,Orange,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,0.1,0,OB,1


In [41]:
# one-hot-encoding columns 

zone_encoding = pd.get_dummies(features["zone_measures"])
acq_encoding = pd.get_dummies(features["acquisition_group"])

# merge with original set 

features = pd.merge(features, acq_encoding, left_index=True, right_index=True)
features = pd.merge(features, zone_encoding, left_index=True, right_index=True)

# drop old columns 

del features["zone_measures"]
del features["acquisition_group"]

features

,holiday,weekend,city,daily_mean,total_precipitation,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,outbreak_related,CC,MISSING INFORMATION,NO KNOWN EPI LINK,OB,TRAVEL,UNSPECIFIED EPI LINK,Grey,Orange,Red
0,0,0,0,4.4,5.2,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,0,1,0,0,0,0,0,0,1,0
1,0,0,0,4.4,5.2,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,0,1,0,0,0,0,0,0,1,0
2,0,0,0,4.4,5.2,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,0,1,0,0,0,0,0,0,1,0
3,0,0,0,4.4,5.2,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,0,1,0,0,0,0,0,0,1,0
4,0,0,0,4.4,5.2,0,0,50,100,-41.0,-14.0,-18.0,-56.0,-48.0,18.0,0.2,1,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,1,1,0,0.7,0.0,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,0.1,0,1,0,0,0,1,0,0,0,1,0
76452,1,1,0,0.7,0.0,0,0,50,100,-31.0,-2.0,47.0,-36.0,-8.0,6.0,0.1,0,1,0,0,0,1,0,0,0,1,0
76453,1,1,1,-1.0,0.0,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,0.1,0,1,0,0,0,1,0,0,0,1,0
76454,1,1,1,-1.0,0.0,0,0,50,100,-26.0,3.0,51.0,-47.0,-6.0,6.0,0.1,0,1,0,0,0,1,0,0,0,1,0


In [42]:
# normalize all numerical columns using min-max scaling 

scaler = MinMaxScaler()

features[["daily_mean", "total_precipitation", "max_indoor_capacity", "max_outdoor_capacity", 
          "retail_and_recreation", "grocery_and_pharmacy", "parks", "transit_stations", "workplaces", 
          "residential"]] = scaler.fit_transform(features[["daily_mean", "total_precipitation", 
                                                                         "max_indoor_capacity","max_outdoor_capacity",
                                                                         "retail_and_recreation", "grocery_and_pharmacy", 
                                                                         "parks", "transit_stations", "workplaces", "residential"]])

features

,holiday,weekend,city,daily_mean,total_precipitation,lockdown,stay_at_home_order,max_indoor_capacity,max_outdoor_capacity,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,age,gender,outbreak_related,CC,MISSING INFORMATION,NO KNOWN EPI LINK,OB,TRAVEL,UNSPECIFIED EPI LINK,Grey,Orange,Red
0,0,0,0,0.664773,0.202335,0,0,1.0,1.0,0.5875,0.560976,0.174359,0.574074,0.431818,0.483871,0.2,1,0,1,0,0,0,0,0,0,1,0
1,0,0,0,0.664773,0.202335,0,0,1.0,1.0,0.5875,0.560976,0.174359,0.574074,0.431818,0.483871,0.2,1,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0.664773,0.202335,0,0,1.0,1.0,0.5875,0.560976,0.174359,0.574074,0.431818,0.483871,0.2,1,0,1,0,0,0,0,0,0,1,0
3,0,0,0,0.664773,0.202335,0,0,1.0,1.0,0.5875,0.560976,0.174359,0.574074,0.431818,0.483871,0.2,1,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0.664773,0.202335,0,0,1.0,1.0,0.5875,0.560976,0.174359,0.574074,0.431818,0.483871,0.2,1,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,1,1,0,0.559659,0.000000,0,0,1.0,1.0,0.7125,0.658537,0.507692,0.944444,0.886364,0.096774,0.1,0,1,0,0,0,1,0,0,0,1,0
76452,1,1,0,0.559659,0.000000,0,0,1.0,1.0,0.7125,0.658537,0.507692,0.944444,0.886364,0.096774,0.1,0,1,0,0,0,1,0,0,0,1,0
76453,1,1,1,0.511364,0.000000,0,0,1.0,1.0,0.7750,0.699187,0.528205,0.740741,0.909091,0.096774,0.1,0,1,0,0,0,1,0,0,0,1,0
76454,1,1,1,0.511364,0.000000,0,0,1.0,1.0,0.7750,0.699187,0.528205,0.740741,0.909091,0.096774,0.1,0,1,0,0,0,1,0,0,0,1,0


In [43]:
# get labels, remove from features set

labels = features["outbreak_related"]
del features["outbreak_related"]

labels

0        0
1        0
2        0
3        0
4        0
        ..
76451    1
76452    1
76453    1
76454    1
76455    1
Name: outbreak_related, Length: 76062, dtype: int64

In [44]:
# get feature values 

feature_values = features.values

feature_values

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.]])

In [45]:
# initial number of features

len(feature_values[1])

26

In [46]:
# feature selection (low variance method)

selector = VarianceThreshold(threshold=(.8*(1-.8)))
selector.fit_transform(feature_values)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [47]:
# number of features after feature selection

len(feature_values[1])

26

In [48]:
# try another method of feature selection (tree-based)

feature_values.shape

(76062, 26)

In [49]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(feature_values, labels)
model = SelectFromModel(clf, prefit=True)
feature_idx = model.get_support()
feature_name = features.columns[feature_idx]
feature_values_new = model.transform(feature_values)
feature_values_new.shape

(76062, 5)

In [50]:
feature_values_new

array([[0.2, 1. , 0. , 0. , 0. ],
       [0.2, 1. , 0. , 0. , 0. ],
       [0.2, 1. , 0. , 0. , 0. ],
       ...,
       [0.1, 0. , 0. , 0. , 1. ],
       [0.1, 0. , 0. , 0. , 1. ],
       [0.1, 0. , 0. , 0. , 1. ]])

In [51]:
feature_name

Index(['age', 'CC', 'MISSING INFORMATION', 'NO KNOWN EPI LINK', 'OB'], dtype='object')

In [52]:
# split into train and test sets (stratified)

x_train, x_test, y_train, y_test = train_test_split(feature_values, labels, test_size = 0.2, shuffle = True, stratify = labels)

In [53]:
Counter(y_train)

Counter({0: 50574, 1: 10275})

In [54]:
Counter(y_test)

Counter({1: 2569, 0: 12644})